# 获取数据路径及文件

In [1]:
import os
all_file_path = []
for root, dirs, files in os.walk('./data'):
    for _dir in dirs:
        all_file_path.append(os.path.join(root, _dir, 'neg.txt'))
        all_file_path.append(os.path.join(root, _dir, 'pos.txt'))
all_file_path

['./data\\clothing\\neg.txt',
 './data\\clothing\\pos.txt',
 './data\\fruit\\neg.txt',
 './data\\fruit\\pos.txt',
 './data\\hotel\\neg.txt',
 './data\\hotel\\pos.txt',
 './data\\pda\\neg.txt',
 './data\\pda\\pos.txt',
 './data\\shampoo\\neg.txt',
 './data\\shampoo\\pos.txt']

# 将数据集中的所有物品类型的评论整合到一起

In [2]:
neg_datalist = []
pos_datalist = []
for file in all_file_path:
    if (os.path.basename(file) == 'neg.txt'):
        with open(file, 'r', encoding='utf-8') as f:
            neg_data = f.read()
            neg_datalist.extend(neg_data.split('\n'))
    else:
        with open(file, 'r', encoding='utf-8') as f:
            pos_data = f.read()
            pos_datalist.extend(pos_data.split('\n'))
len(neg_datalist)

25000

In [3]:
import numpy as np

dataset = np.array(neg_datalist+pos_datalist)
labels = np.array([0]*len(neg_datalist)+[1]*len(pos_datalist))
print(len(dataset))

50000


In [4]:
np.random.seed(2022)
mix_index = np.random.choice(len(dataset),len(dataset))
dataset = dataset[mix_index]
labels = labels[mix_index]

print(labels)

[0 0 0 ... 0 1 1]


# 划分训练集和测试集

In [5]:
from sklearn.model_selection import train_test_split

train_samples, eval_samples, train_labels, eval_labels = train_test_split(
    dataset, labels, test_size=0.1, shuffle=True, random_state=2022)
train_samples = train_samples.tolist()
eval_samples = eval_samples.tolist()
print(len(train_samples))
print(len(eval_samples))

45000
5000


# 得到标签的One-hot编码

In [6]:
def get_dummies(l, size=2):
    res = list()
    for i in l:
        tmp = [0]*size
        tmp[i] = 1
        res.append(tmp)
    return res

# 定义Bert模型

In [7]:
import torch
from transformers import BertTokenizer
from torch.utils.data import DataLoader, TensorDataset

model_name_path = './pretrain_model/bert-base-chinese/'

tokenizer = BertTokenizer.from_pretrained(model_name_path)

In [8]:
# tokenized_text = [tokenizer.tokenize(i) for i in train_sample]
# input_ids = [tokenizer.convert_tokens_to_ids(i) for i in tokenized_text]
# input_labels = get_dummies(train_labels)
# input_ids[0], input_labels[0]

In [9]:
# for i in range(len(input_ids)):
#     # 将数据样本填充至长度为512
#     seq = input_ids[i]
#     if len(seq) != 512:
#         input_ids[i].extend([0]*(512-len(seq)))

# 词元化文本

In [10]:
model_train_inputs = tokenizer(train_samples,
                               padding=True,
                               truncation=True,
                               max_length=512,
                               return_tensors='pt')

# 转换标签

In [11]:
input_labels = get_dummies(train_labels)

# 构建训练和测试的数据集

In [12]:
train_set = TensorDataset(model_train_inputs['input_ids'], torch.FloatTensor(input_labels))

In [13]:
train_loader = DataLoader(dataset=train_set, batch_size=4, shuffle=True)
train_loader

In [14]:
model_eval_inputs = tokenizer(eval_samples,
                              padding=True,
                              truncation=True,
                              max_length=512,
                              return_tensors='pt')
input_labels = get_dummies(eval_labels)

eval_set = TensorDataset(model_eval_inputs['input_ids'],
                         torch.FloatTensor(input_labels))
eval_loader = DataLoader(dataset=eval_set, batch_size=1, shuffle=True)
eval_loader

# 定义训练的设备，GPU or CPU

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# 搭建Bert分类模型

In [16]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel


class fn_cls(nn.Module):
    def __init__(self):
        super(fn_cls, self).__init__()
        self.model = BertModel.from_pretrained(model_name_path, cache_dir="./")
        self.model.to(device)
        self.dropout = nn.Dropout(0.1)
        self.l1 = nn.Linear(768, 2)

    def forward(self, x, attention_mask=None):
        outputs = self.model(x, attention_mask=attention_mask)
        x = outputs[1]  # 取池化后的结果 batch * 768
        x = x.view(-1, 768)
        x = self.dropout(x)
        x = self.l1(x)
        return x


# 定义优化器，损失函数等

In [17]:
from torch import optim

cls = fn_cls()
cls.to(device)
cls.train()

criterion = nn.BCELoss()
sigmoid = nn.Sigmoid()
optimizer = optim.Adam(cls.parameters(), lr=1e-5)


Some weights of the model checkpoint at ./pretrain_model/bert-base-chinese/ were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# 定义预测函数

In [18]:
def predict(logits):
    res = torch.argmax(logits, 1)
    return res


# 训练过程 fine-tuning

In [19]:
from torch.autograd import Variable
import time

pre = time.time()

accumulation_steps = 8
epoch = 2

for i in range(epoch):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).to(device), Variable(
            target.view(-1, 2)).to(device)

        mask = []
        for sample in data:
            mask.append([1 if i != 0 else 0 for i in sample])
        mask = torch.Tensor(mask).to(device)
        
        output = cls(data, attention_mask=mask)
        pred = predict(output)

        loss = criterion(sigmoid(output).view(-1, 2), target)

        # 梯度积累
        loss = loss/accumulation_steps
        loss.backward()

        if((batch_idx+1) % accumulation_steps) == 0:
            # 每 8 次更新一下网络中的参数
            optimizer.step()
            optimizer.zero_grad()

        if ((batch_idx+1) % accumulation_steps) == 1:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss:{:.6f}'.format(
                i+1, batch_idx, len(train_loader), 100. *
                batch_idx/len(train_loader), loss.item()
            ))
        if batch_idx == len(train_loader)-1:
            # 在每个 Epoch 的最后输出一下结果
            print('labels:', target)
            print('pred:', pred)

print('训练时间：', time.time()-pre)


Train Epoch: 1 [0/11250 (0%)]	Loss:0.074119
Train Epoch: 1 [8/11250 (0%)]	Loss:0.097051
Train Epoch: 1 [16/11250 (0%)]	Loss:0.084498
Train Epoch: 1 [24/11250 (0%)]	Loss:0.072759
Train Epoch: 1 [32/11250 (0%)]	Loss:0.091817
Train Epoch: 1 [40/11250 (0%)]	Loss:0.075639
Train Epoch: 1 [48/11250 (0%)]	Loss:0.068690
Train Epoch: 1 [56/11250 (0%)]	Loss:0.083651
Train Epoch: 1 [64/11250 (1%)]	Loss:0.068478
Train Epoch: 1 [72/11250 (1%)]	Loss:0.085388
Train Epoch: 1 [80/11250 (1%)]	Loss:0.083749
Train Epoch: 1 [88/11250 (1%)]	Loss:0.066725
Train Epoch: 1 [96/11250 (1%)]	Loss:0.063587
Train Epoch: 1 [104/11250 (1%)]	Loss:0.070430
Train Epoch: 1 [112/11250 (1%)]	Loss:0.076103
Train Epoch: 1 [120/11250 (1%)]	Loss:0.069125
Train Epoch: 1 [128/11250 (1%)]	Loss:0.059819
Train Epoch: 1 [136/11250 (1%)]	Loss:0.053823
Train Epoch: 1 [144/11250 (1%)]	Loss:0.066529
Train Epoch: 1 [152/11250 (1%)]	Loss:0.047494
Train Epoch: 1 [160/11250 (1%)]	Loss:0.040653
Train Epoch: 1 [168/11250 (1%)]	Loss:0.036640
Tra

Train Epoch: 1 [1416/11250 (13%)]	Loss:0.004881
Train Epoch: 1 [1424/11250 (13%)]	Loss:0.035468
Train Epoch: 1 [1432/11250 (13%)]	Loss:0.080042
Train Epoch: 1 [1440/11250 (13%)]	Loss:0.107288
Train Epoch: 1 [1448/11250 (13%)]	Loss:0.002668
Train Epoch: 1 [1456/11250 (13%)]	Loss:0.024738
Train Epoch: 1 [1464/11250 (13%)]	Loss:0.003247
Train Epoch: 1 [1472/11250 (13%)]	Loss:0.030875
Train Epoch: 1 [1480/11250 (13%)]	Loss:0.139344
Train Epoch: 1 [1488/11250 (13%)]	Loss:0.033692
Train Epoch: 1 [1496/11250 (13%)]	Loss:0.065165
Train Epoch: 1 [1504/11250 (13%)]	Loss:0.108597
Train Epoch: 1 [1512/11250 (13%)]	Loss:0.008621
Train Epoch: 1 [1520/11250 (14%)]	Loss:0.073955
Train Epoch: 1 [1528/11250 (14%)]	Loss:0.009114
Train Epoch: 1 [1536/11250 (14%)]	Loss:0.005742
Train Epoch: 1 [1544/11250 (14%)]	Loss:0.009448
Train Epoch: 1 [1552/11250 (14%)]	Loss:0.005788
Train Epoch: 1 [1560/11250 (14%)]	Loss:0.013314
Train Epoch: 1 [1568/11250 (14%)]	Loss:0.008363
Train Epoch: 1 [1576/11250 (14%)]	Loss:0

Train Epoch: 1 [2784/11250 (25%)]	Loss:0.022754
Train Epoch: 1 [2792/11250 (25%)]	Loss:0.028967
Train Epoch: 1 [2800/11250 (25%)]	Loss:0.003604
Train Epoch: 1 [2808/11250 (25%)]	Loss:0.010503
Train Epoch: 1 [2816/11250 (25%)]	Loss:0.002375
Train Epoch: 1 [2824/11250 (25%)]	Loss:0.002602
Train Epoch: 1 [2832/11250 (25%)]	Loss:0.012713
Train Epoch: 1 [2840/11250 (25%)]	Loss:0.006872
Train Epoch: 1 [2848/11250 (25%)]	Loss:0.005214
Train Epoch: 1 [2856/11250 (25%)]	Loss:0.116276
Train Epoch: 1 [2864/11250 (25%)]	Loss:0.004473
Train Epoch: 1 [2872/11250 (26%)]	Loss:0.010488
Train Epoch: 1 [2880/11250 (26%)]	Loss:0.004611
Train Epoch: 1 [2888/11250 (26%)]	Loss:0.004890
Train Epoch: 1 [2896/11250 (26%)]	Loss:0.009082
Train Epoch: 1 [2904/11250 (26%)]	Loss:0.040416
Train Epoch: 1 [2912/11250 (26%)]	Loss:0.102716
Train Epoch: 1 [2920/11250 (26%)]	Loss:0.006847
Train Epoch: 1 [2928/11250 (26%)]	Loss:0.012722
Train Epoch: 1 [2936/11250 (26%)]	Loss:0.001810
Train Epoch: 1 [2944/11250 (26%)]	Loss:0

Train Epoch: 1 [4152/11250 (37%)]	Loss:0.003093
Train Epoch: 1 [4160/11250 (37%)]	Loss:0.010356
Train Epoch: 1 [4168/11250 (37%)]	Loss:0.004110
Train Epoch: 1 [4176/11250 (37%)]	Loss:0.007100
Train Epoch: 1 [4184/11250 (37%)]	Loss:0.006716
Train Epoch: 1 [4192/11250 (37%)]	Loss:0.008108
Train Epoch: 1 [4200/11250 (37%)]	Loss:0.006064
Train Epoch: 1 [4208/11250 (37%)]	Loss:0.012383
Train Epoch: 1 [4216/11250 (37%)]	Loss:0.084356
Train Epoch: 1 [4224/11250 (38%)]	Loss:0.020880
Train Epoch: 1 [4232/11250 (38%)]	Loss:0.031586
Train Epoch: 1 [4240/11250 (38%)]	Loss:0.093407
Train Epoch: 1 [4248/11250 (38%)]	Loss:0.003482
Train Epoch: 1 [4256/11250 (38%)]	Loss:0.002972
Train Epoch: 1 [4264/11250 (38%)]	Loss:0.002802
Train Epoch: 1 [4272/11250 (38%)]	Loss:0.036389
Train Epoch: 1 [4280/11250 (38%)]	Loss:0.003093
Train Epoch: 1 [4288/11250 (38%)]	Loss:0.004264
Train Epoch: 1 [4296/11250 (38%)]	Loss:0.029180
Train Epoch: 1 [4304/11250 (38%)]	Loss:0.001481
Train Epoch: 1 [4312/11250 (38%)]	Loss:0

Train Epoch: 1 [5520/11250 (49%)]	Loss:0.013352
Train Epoch: 1 [5528/11250 (49%)]	Loss:0.005234
Train Epoch: 1 [5536/11250 (49%)]	Loss:0.050792
Train Epoch: 1 [5544/11250 (49%)]	Loss:0.290714
Train Epoch: 1 [5552/11250 (49%)]	Loss:0.007393
Train Epoch: 1 [5560/11250 (49%)]	Loss:0.001391
Train Epoch: 1 [5568/11250 (49%)]	Loss:0.007310
Train Epoch: 1 [5576/11250 (50%)]	Loss:0.042026
Train Epoch: 1 [5584/11250 (50%)]	Loss:0.018629
Train Epoch: 1 [5592/11250 (50%)]	Loss:0.109533
Train Epoch: 1 [5600/11250 (50%)]	Loss:0.004957
Train Epoch: 1 [5608/11250 (50%)]	Loss:0.006219
Train Epoch: 1 [5616/11250 (50%)]	Loss:0.003602
Train Epoch: 1 [5624/11250 (50%)]	Loss:0.010781
Train Epoch: 1 [5632/11250 (50%)]	Loss:0.105350
Train Epoch: 1 [5640/11250 (50%)]	Loss:0.010905
Train Epoch: 1 [5648/11250 (50%)]	Loss:0.005725
Train Epoch: 1 [5656/11250 (50%)]	Loss:0.009153
Train Epoch: 1 [5664/11250 (50%)]	Loss:0.009558
Train Epoch: 1 [5672/11250 (50%)]	Loss:0.059790
Train Epoch: 1 [5680/11250 (50%)]	Loss:0

...

Train Epoch: 2 [5208/11250 (46%)]	Loss:0.001679
Train Epoch: 2 [5216/11250 (46%)]	Loss:0.002600
Train Epoch: 2 [5224/11250 (46%)]	Loss:0.002831
Train Epoch: 2 [5232/11250 (47%)]	Loss:0.001166
Train Epoch: 2 [5240/11250 (47%)]	Loss:0.002040
Train Epoch: 2 [5248/11250 (47%)]	Loss:0.001955
Train Epoch: 2 [5256/11250 (47%)]	Loss:0.002836
Train Epoch: 2 [5264/11250 (47%)]	Loss:0.004963
Train Epoch: 2 [5272/11250 (47%)]	Loss:0.010919
Train Epoch: 2 [5280/11250 (47%)]	Loss:0.002029
Train Epoch: 2 [5288/11250 (47%)]	Loss:0.002856
Train Epoch: 2 [5296/11250 (47%)]	Loss:0.006192
Train Epoch: 2 [5304/11250 (47%)]	Loss:0.096888
Train Epoch: 2 [5312/11250 (47%)]	Loss:0.027334
Train Epoch: 2 [5320/11250 (47%)]	Loss:0.000623
Train Epoch: 2 [5328/11250 (47%)]	Loss:0.001209
Train Epoch: 2 [5336/11250 (47%)]	Loss:0.001047
Train Epoch: 2 [5344/11250 (48%)]	Loss:0.002239
Train Epoch: 2 [5352/11250 (48%)]	Loss:0.000903
Train Epoch: 2 [5360/11250 (48%)]	Loss:0.003101
Train Epoch: 2 [5368/11250 (48%)]	Loss:0

Train Epoch: 2 [6576/11250 (58%)]	Loss:0.069611
Train Epoch: 2 [6584/11250 (59%)]	Loss:0.000985
Train Epoch: 2 [6592/11250 (59%)]	Loss:0.000805
Train Epoch: 2 [6600/11250 (59%)]	Loss:0.094372
Train Epoch: 2 [6608/11250 (59%)]	Loss:0.002677
Train Epoch: 2 [6616/11250 (59%)]	Loss:0.011457
Train Epoch: 2 [6624/11250 (59%)]	Loss:0.003854
Train Epoch: 2 [6632/11250 (59%)]	Loss:0.003800
Train Epoch: 2 [6640/11250 (59%)]	Loss:0.000669
Train Epoch: 2 [6648/11250 (59%)]	Loss:0.001378
Train Epoch: 2 [6656/11250 (59%)]	Loss:0.000583
Train Epoch: 2 [6664/11250 (59%)]	Loss:0.000456
Train Epoch: 2 [6672/11250 (59%)]	Loss:0.001577
Train Epoch: 2 [6680/11250 (59%)]	Loss:0.004162
Train Epoch: 2 [6688/11250 (59%)]	Loss:0.001243
Train Epoch: 2 [6696/11250 (60%)]	Loss:0.001926
Train Epoch: 2 [6704/11250 (60%)]	Loss:0.000720
Train Epoch: 2 [6712/11250 (60%)]	Loss:0.000659
Train Epoch: 2 [6720/11250 (60%)]	Loss:0.000402
Train Epoch: 2 [6728/11250 (60%)]	Loss:0.005600
Train Epoch: 2 [6736/11250 (60%)]	Loss:0

Train Epoch: 2 [7944/11250 (71%)]	Loss:0.000356
Train Epoch: 2 [7952/11250 (71%)]	Loss:0.003506
Train Epoch: 2 [7960/11250 (71%)]	Loss:0.004434
Train Epoch: 2 [7968/11250 (71%)]	Loss:0.007642
Train Epoch: 2 [7976/11250 (71%)]	Loss:0.005462
Train Epoch: 2 [7984/11250 (71%)]	Loss:0.000695
Train Epoch: 2 [7992/11250 (71%)]	Loss:0.007020
Train Epoch: 2 [8000/11250 (71%)]	Loss:0.001555
Train Epoch: 2 [8008/11250 (71%)]	Loss:0.001115
Train Epoch: 2 [8016/11250 (71%)]	Loss:0.003887
Train Epoch: 2 [8024/11250 (71%)]	Loss:0.001470
Train Epoch: 2 [8032/11250 (71%)]	Loss:0.004413
Train Epoch: 2 [8040/11250 (71%)]	Loss:0.003528
Train Epoch: 2 [8048/11250 (72%)]	Loss:0.000402
Train Epoch: 2 [8056/11250 (72%)]	Loss:0.012024
Train Epoch: 2 [8064/11250 (72%)]	Loss:0.002352
Train Epoch: 2 [8072/11250 (72%)]	Loss:0.010469
Train Epoch: 2 [8080/11250 (72%)]	Loss:0.002128
Train Epoch: 2 [8088/11250 (72%)]	Loss:0.002658
Train Epoch: 2 [8096/11250 (72%)]	Loss:0.004838
Train Epoch: 2 [8104/11250 (72%)]	Loss:0

Train Epoch: 2 [9312/11250 (83%)]	Loss:0.001960
Train Epoch: 2 [9320/11250 (83%)]	Loss:0.003043
Train Epoch: 2 [9328/11250 (83%)]	Loss:0.001475
Train Epoch: 2 [9336/11250 (83%)]	Loss:0.000636
Train Epoch: 2 [9344/11250 (83%)]	Loss:0.000993
Train Epoch: 2 [9352/11250 (83%)]	Loss:0.002126
Train Epoch: 2 [9360/11250 (83%)]	Loss:0.001844
Train Epoch: 2 [9368/11250 (83%)]	Loss:0.000548
Train Epoch: 2 [9376/11250 (83%)]	Loss:0.010471
Train Epoch: 2 [9384/11250 (83%)]	Loss:0.002004
Train Epoch: 2 [9392/11250 (83%)]	Loss:0.000746
Train Epoch: 2 [9400/11250 (84%)]	Loss:0.005172
Train Epoch: 2 [9408/11250 (84%)]	Loss:0.000964
Train Epoch: 2 [9416/11250 (84%)]	Loss:0.000291
Train Epoch: 2 [9424/11250 (84%)]	Loss:0.001805
Train Epoch: 2 [9432/11250 (84%)]	Loss:0.008487
Train Epoch: 2 [9440/11250 (84%)]	Loss:0.002950
Train Epoch: 2 [9448/11250 (84%)]	Loss:0.000659
Train Epoch: 2 [9456/11250 (84%)]	Loss:0.022436
Train Epoch: 2 [9464/11250 (84%)]	Loss:0.001708
Train Epoch: 2 [9472/11250 (84%)]	Loss:0

Train Epoch: 2 [10664/11250 (95%)]	Loss:0.000859
Train Epoch: 2 [10672/11250 (95%)]	Loss:0.002256
Train Epoch: 2 [10680/11250 (95%)]	Loss:0.000527
Train Epoch: 2 [10688/11250 (95%)]	Loss:0.000986
Train Epoch: 2 [10696/11250 (95%)]	Loss:0.001796
Train Epoch: 2 [10704/11250 (95%)]	Loss:0.006070
Train Epoch: 2 [10712/11250 (95%)]	Loss:0.000480
Train Epoch: 2 [10720/11250 (95%)]	Loss:0.004360
Train Epoch: 2 [10728/11250 (95%)]	Loss:0.000593
Train Epoch: 2 [10736/11250 (95%)]	Loss:0.005270
Train Epoch: 2 [10744/11250 (96%)]	Loss:0.001826
Train Epoch: 2 [10752/11250 (96%)]	Loss:0.000562
Train Epoch: 2 [10760/11250 (96%)]	Loss:0.000830
Train Epoch: 2 [10768/11250 (96%)]	Loss:0.001030
Train Epoch: 2 [10776/11250 (96%)]	Loss:0.001067
Train Epoch: 2 [10784/11250 (96%)]	Loss:0.004393
Train Epoch: 2 [10792/11250 (96%)]	Loss:0.000569
Train Epoch: 2 [10800/11250 (96%)]	Loss:0.002298
Train Epoch: 2 [10808/11250 (96%)]	Loss:0.000909
Train Epoch: 2 [10816/11250 (96%)]	Loss:0.020410
Train Epoch: 2 [1082

# 测试过程

In [20]:
from tqdm import tqdm_notebook as tqdm

cls.eval()

correct = 0
total = 0

for batch_idx, (data, target) in enumerate(tqdm(eval_loader)):
    data = data.to(device)
    target = target.long().to(device)

    mask = []
    for sample in data:
        mask.append([1 if i != 0 else 0 for i in sample])
    mask = torch.Tensor(mask).to(device)

    output = cls(data, attention_mask=mask)
    pred = predict(output)

    correct += (pred == target).sum().item()
    total += len(data)

# 准确率应该达到百分之 90 以上
print('正确分类的样本数：{}，样本总数：{}，准确率：{:.2f}%'.format(
    correct, total, 100.*correct/total))


<ipython-input-20-f19ed8d01f1e>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_idx, (data, target) in enumerate(tqdm(eval_loader)):


  0%|          | 0/5000 [00:00<?, ?it/s]

正确分类的样本数：5000，样本总数：5000，准确率：100.00%


In [46]:
test_samples = ['很棒的购物体验，但是东西有些问题，物流可以，但是东西没有吊牌', '物流还是挺快的，很好']

In [47]:
model_inputs = tokenizer(test_samples,
                               padding=True,
                               truncation=True,
                               max_length=512,
                               return_tensors='pt')
model_inputs

{'input_ids': tensor([[ 101, 2523, 3472, 4638, 6579, 4289,  860, 7741, 8024,  852, 3221,  691,
         6205, 3300,  763, 7309, 7579, 8024, 4289, 3837, 1377,  809, 8024,  852,
         3221,  691, 6205, 3766, 3300, 1396, 4277,  102],
        [ 101, 4289, 3837, 6820, 3221, 2923, 2571, 4638, 8024, 2523, 1962,  102,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])}

In [48]:
cls.eval()

output = cls(model_inputs['input_ids'].to(device), attention_mask=model_inputs['attention_mask'].to(device))
pred = predict(output)
pred


tensor([1, 1], device='cuda:0')